# Assignment: Data Wrangling
## `! git clone https://github.com/DS3001/wrangling`
## Do Q2, and one of Q1 or Q3.

**Q1.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about?
  2. Read the introduction. What is the "tidy data standard" intended to accomplish?
  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."
  4. Read Section 2.2. How does Wickham define values, variables, and observations?
  5. How is "Tidy Data" defined in section 2.3?
  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?
  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?
  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?

**Q2.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the `./data/sharks.csv` data covered in the lecture, clean the "Type" variable as well as you can, and explain the choices you make.
3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

Q2.1

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

First, I inspected the data frame using the .head() method and the .shape attribute to better visualize the data. Next, I printed the data type of the 'Price' variable and saw that it was of type 'object', which we do not want. Then, I printed all the unique values for 'Price' and saw that many were comma separated, which causes those values to be read in as strings. 

In [ ]:
df = pd.read_csv('./data/airbnb_hw.csv', low_memory=False)
df.head()
var = 'Price'
print('Shape: ', df.shape, '\n') # 30478 rows, 13 columns
print('Price Type (Before Cleaning): ', df[var].dtypes, '\n') # Type 'object'
print('Unique Values (Before Cleaning): ', df[var].unique(), '\n')

To clean the data, I removed commas from all the values using the .replace() method to prepare them to be converted into numeric values. Then, I used panda's .to_numeric() method to type cast all the values to integers and confirmed that this change was applied using the .dtype attribute and the .unique() method. Finally, I added all the null 'Price' values and found that there were 0 missing values.

In [ ]:
df[var] = df[var].str.replace(',', '') # Remove commas from all values
df[var] = pd.to_numeric(df[var], errors="coerce") # Type casting all values to int
print('Price Type (After Cleaning): ', df[var].dtype, '\n') # Confirm data type has changed to int
print('Unique Values (After Cleaning): ', df[var].unique(), '\n')

print('Total Missing: ', sum(df[var].isnull()), '\n') # 0 missing values

Q2.2

First, I inspected the data frame using the .unique() method and found that there was a nan category, and there were similar categories that could be consolidated under a single category (e.g. invalid, questionable, unconfirmed, unverified, and under investigation).

In [ ]:
sdf = pd.read_csv('./data/sharks.csv', low_memory=False)
sdf.head()
print('Unique Values: ', sdf['Type'].unique(), '\n') # nan value(s)
print(sdf['Type'].value_counts(), '\n') # Boat vs. boating vs. boatomg, invalid vs. unconfirmed vs. unverified vs. questionable vs. under investigation

I created a dummy variable, case_type, to hold the cleaned categories. I used the .replace() method to consolidate the 'uncertain' data (i.e. invalid, unconfirmed, unverified, questionable, and under investigation) under the nan category (i.e. treat them as missing), as well as consolidate the sea-related events under the category 'Sea Disaster'.

In [ ]:
case_type = sdf['Type']
case_type = case_type.replace(['Invalid', 'Questionable', 'Unconfirmed', 'Unverified', 'Under investigation'], np.nan)
case_type = case_type.replace(['Watercraft', 'Boat', 'Boating', 'Boatomg'], 'Sea Disaster')

sdf['Type'] = case_type
del case_type

print('Unique Values: ', sdf['Type'].unique(), '\n')
print(sdf['Type'].value_counts(), '\n')

Q2.3: Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.

In [ ]:
pdf = pd.read_parquet('./data/justice_data.parquet', 'pyarrow') # pip installed pyarrow
pdf.head()
var = 'WhetherDefendantWasReleasedPretrial'
print(pdf[var].dtype, '\n') # integer
print(pdf[var].value_counts(), '\n') # 9 not defined in codebook
print('Total Missing (Before Cleaning): ', sum(pdf[var].isnull()), '\n')

released = pdf[var]
released = released.replace(9, np.nan)
pdf[var] = released
del released
print(pdf[var].value_counts(), '\n')
print('Total Missing (After Cleaning): ', sum(pdf[var].isnull()))

**Q3.** Many important datasets contain a race variable, typically limited to a handful of values often including Black, White, Asian, Latino, and Indigenous. This question looks at data gathering efforts on this variable by the U.S. Federal government.

1. How did the most recent US Census gather data on race?
2. Why do we gather these data? What role do these kinds of data play in politics and society? Why does data quality matter?
3. Please provide a constructive criticism of how the Census was conducted: What was done well? What do you think was missing? How should future large scale surveys be adjusted to best reflect the diversity of the population? Could some of the Census' good practices be adopted more widely to gather richer and more useful data?
4. How did the Census gather data on sex and gender? Please provide a similar constructive criticism of their practices.
5. When it comes to cleaning data, what concerns do you have about protected characteristics like sex, gender, sexual identity, or race? What challenges can you imagine arising when there are missing values? What good or bad practices might people adopt, and why?
6. Suppose someone invented an algorithm to impute values for protected characteristics like race, gender, sex, or sexuality. What kinds of concerns would you have?